In [4]:
import pandas as pd
import geopandas as gpd
import os
from joblib import Parallel, delayed
from pathlib import Path
from proxi_API.data.settings import N_CORES, H3_ZOOM
from proxi_API.model.data_processing import get_city, get_streets
from proxi_API.model import data_aggregation
from proxi_API.model import mobility_indices
from proxi_API.model import h3_mapping
import osmnx as ox
import logging

In [8]:
CITY = 'Barcelona'
out = Path("src/proxi_API/data/cities")
agg = gpd.read_file(out / f"{CITY}_{H3_ZOOM}_agg.geojson")

In [9]:
agg.keys()

Index(['proximity_time_foot', 'residentes_total', 'visita_tur_stica_total',
       'trabajadores_estudiantes_total', 'compras_ocio_total',
       'acceso_hosteler_a_total', 'acceso_tpte_p_blico_total', 'p_t',
       'geoid_left', 'mob_index', 'residentes_index', 'turistas_index',
       'trabajadores_index', 'compras_index', 'hosteleria_index',
       'transporte_index', 'h3_id', 'geometry'],
      dtype='object')

In [10]:
agg.p_t.sum()

np.float64(7515453.533177729)

In [12]:
agg.mob_index.sum()

np.float64(0.21599756255680175)